单个case的修改

In [1]:
import sys, glob, os, argparse
import SimpleITK as sitk
import pandas as pd
import shutil
import pprint, pickle
import numpy as np
from skimage import measure
import math
from utils import *

In [ ]:
pkl_file = './after_headcut_properties_CellPressAll.pkl'
prediction = './Tr0214.nii.gz'

In [4]:
small_IAs_num, middle_IAs_num, large_IAs_num = 0,0,0

patientID = os.path.basename(prediction)[:-7]

print (patientID)

pred_nii = sitk.ReadImage(prediction)
pred_np = sitk.GetArrayFromImage(pred_nii).astype(np.int32)

pred_bin_np = pred_np > 0
pred_lbls_np = measure.label(pred_bin_np)
labels = np.unique(pred_lbls_np)

if len(labels) == 1 and labels[0] == 0:
    shutil.copyfile(prediction,os.path.join(args.ori_folder,'multilabel',patientID+'.nii.gz'))


elif len(labels) > 1:
    labels = labels[1:]

    new_multilabel_save_np = np.zeros_like(pred_lbls_np)

    space_prod = np.prod(propertes[patientID]['original_spacing'])

    diameters = []
    voxels = []
    for lbl in labels:
        pred_lbl_np = np.zeros_like(pred_lbls_np)
        lbl_voxels = np.sum(pred_lbls_np == lbl)
        voxels.append(lbl_voxels)
        diameter = 2*(((space_prod*lbl_voxels)*(3/(4*math.pi)))**(1/3))
        diameters.append(diameter)
        assert diameter > 0

        if diameter < 5:
            new_multilabel_save_np[pred_lbls_np == lbl] = 1
            small_IAs_num += 1

        elif (diameter >= 5) and (diameter < 15):
            new_multilabel_save_np[pred_lbls_np == lbl] = 2
            middle_IAs_num += 1

        elif diameter >= 15:
            new_multilabel_save_np[pred_lbls_np == lbl] = 3
            large_IAs_num += 1

    new_multilabel_nii = save_itk_from_numpy(new_multilabel_save_np, propertes[patientID])
    sitk.WriteImage(new_multilabel_nii, os.path.join(args.ori_folder,'multilabel',patientID+'.nii.gz'))


OrderedDict([('id', 'Ts0001'),
             ('ori_cta', 'F:/data/CellPress1338\\cta_img\\Ts0001.nii.gz'),
             ('ori_seg', 'F:/data/CellPress1338\\ane_seg\\Ts0001.nii.gz'),
             ('original_size_of_raw_data', array([693, 512, 512])),
             ('original_spacing', array([0.70001221, 0.62695312, 0.62695312])),
             ('itk_origin',
              (-156.1865234375, -310.1865234375, -411.20001220703125)),
             ('itk_spacing', (0.626953125, 0.626953125, 0.70001220703125)),
             ('itk_direction', (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)),
             ('slices', 693),
             ('IA_voxels', 0),
             ('coords', (559, 693, 83, 444, 83, 444)),
             ('before_size', (693, 512, 512)),
             ('after_size', (134, 361, 361)),
             ('just_head_cta_save',
              'F:/data/CellPress1338/headcut_all\\cta_img\\Ts0001.nii.gz'),
             ('just_head_seg_save',
              'F:/data/CellPress1338/headcut_all\\ane_seg\\